In [1]:
from sklearn.linear_model import LogisticRegression
import tpot2
import numpy as np
import copy
from tqdm import tqdm, tnrange, tqdm_notebook
import sklearn
import sklearn.metrics
import sklearn.datasets
import tpot2 

def params_LogisticRegression(trial, name=None):
    params = {}
    params['solver'] = trial.suggest_categorical(name=f'solver_{name}',
                                                 choices=[f'newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'])
    params['dual'] = False
    params['penalty'] = 'l2'
    params['C'] = trial.suggest_float(f'C_{name}', 1e-4, 1e4, log=True)
    params['l1_ratio'] = None
    if params['solver'] == 'liblinear':
        params['penalty'] = trial.suggest_categorical(name=f'penalty_{name}', choices=['l1', 'l2'])
        if params['penalty'] == 'l2':
            params['dual'] = trial.suggest_categorical(name=f'dual_{name}', choices=[True, False])
        else:
            params['penalty'] = 'l1'

    params['class_weight'] = trial.suggest_categorical(name=f'class_weight_{name}', choices=['balanced'])
    param_grid = {'solver': params['solver'],
                  'penalty': params['penalty'],
                  'dual': params['dual'],
                  'multi_class': 'auto',
                  'l1_ratio': params['l1_ratio'],
                  'C': params['C'],
                  'n_jobs': 1,
                  }
    return param_grid



import tpot2
import sklearn
import sklearn.datasets

est = tpot2.TPOTEstimatorSteadyState(  population_size=10,
                                        initial_population_size = 10,
                                        max_queue_size = 10,
                                        min_individuals_finished = 1,
                                        max_evaluated_individuals = 50,
                            scorers=['roc_auc_ovr',tpot2.model_complexity_scorer], #scorers can be a list of strings or a list of scorers. These get evaluated during cross validation. 
                            scorers_weights=[1,-1],
                            classification=True,
                            root_config_dict =  tpot2.make_classifier_config_dictionary(),
                            #inner_config_dict= "arithmetic_transformer",
                            #leaf_config_dict="feature_set_selector",
                            n_jobs=10,

                            early_stop=None, #how many generations with no improvement to stop after
                            
                            #List of other objective functions. All objective functions take in an untrained GraphPipeline and return a score or a list of scores
                            other_objective_functions= [],
                            
                            #List of weights for the other objective functions. Must be the same length as other_objective_functions. By default, bigger is better is set to True. 
                            other_objective_functions_weights=[],
                            processes=True,
                            verbose=1,
                            max_eval_time_seconds=30,
                            max_time_seconds=600,
                            )

scorer = sklearn.metrics.get_scorer('roc_auc_ovo')
X, y = sklearn.datasets.load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, train_size=0.75, test_size=0.25)
est.fit(X_train, y_train)
print(scorer(est, X_test, y_test))

Evaluations: 100%|██████████| 50/50 [00:38<00:00,  1.32it/s]


FOO!
0.9998370669115155
